In [370]:
from copy import deepcopy
from time import time
from typing import List, Type, Set
from sortedcontainers import SortedListWithKey, SortedSet

step_cost = {0: 1, 1: 10, 2: 100, 3: 1000}
names_to_indices = {'A': 0, 'B': 1, 'C': 2, 'D': 3}
rooms_positions = {0:2, 1:4, 2:6, 3:8}

class Burrow:
    def __init__(self, rooms):
        self.hallway = 11 * [None]
        self.rooms = rooms
        self.cost = 0
        self.prev = None
        
    def __repr__(self) -> str:
        #return f"{{hallway: {self.hallway},\n rooms: {self.rooms}\n cost: {self.cost}}}"
        lines = []
        hallway_str = "".join(list(map(lambda x: '.' if x is None else str(x),list(self.hallway))))
        lines.append("#############")
        lines.append(f"#{hallway_str}#")
        for idx in range(len(self.rooms[0])-1, -1, -1):
            lines.append(f"###{self.rooms[0][idx] if self.rooms[0][idx] is not None else '.'}#{self.rooms[1][idx] if self.rooms[1][idx] is not None else '.'}#{self.rooms[2][idx] if self.rooms[2][idx] is not None else '.'}#{self.rooms[3][idx] if self.rooms[3][idx] is not None else '.'}###")
        lines.append(f"  ######### ")
        lines.append(f"cost: {self.cost}")
        return "\n".join(lines)

    def __hash__(self) -> int:
        return hash((tuple(self.hallway), tuple(map(tuple, self.rooms))))

    def __eq__(self, __o: object) -> bool:
        return hash(self)==hash(__o)
        
    def _reachable_hallway_positions(self, room_n):
        res = []
        for i in range(rooms_positions[room_n], -1, -1):
            if i not in rooms_positions.values():
                if self.hallway[i] is None:
                    res.append(i)
                else: 
                    break

        for i in range(rooms_positions[room_n], len(self.hallway)):
            if i not in rooms_positions.values():
                if self.hallway[i] is None:
                    res.append(i)
                else: 
                    break
        return res

    def room_is_available(self, ap):
        return (self.rooms[ap][-1] is None and all([p==ap for p in self.rooms[ap] if p is not None]))

    def path_is_clear(self, from_idx, to_idx, ap_in_hallway=False):
        f = min(from_idx, to_idx)
        t = max(from_idx, to_idx)
        if ap_in_hallway:
            if from_idx==f: 
                f += 1
            else:
                t -= 1
        if f==t:
            return True
        return all([pos is None for pos in self.hallway[f:t+1]])
    
    def _add_scenario(self, new_scenario : "Type[Burrow]", this_step_cost, open_scenarios, closed_scenarios):
        new_scenario.cost = self.cost + this_step_cost
        #if new_scenario.is_end():
        #    print(f"adding new end scenario with cost: {new_scenario.cost}")
        new_scenario.prev = self
        new_scenario.update_scenario(open_scenarios, closed_scenarios)
        #print(new_scenario)
        

    def add_possible_outcomes(self, open_scenarios : "List[Type[Burrow]]", closed_scenarios: "List[Type[Burrow]]"):
        for i, room in enumerate(self.rooms):
            occupied = list(filter(lambda p: p is not None, room))
            if len(occupied)>0:
                ap = occupied[-1]
                ap_pos = rooms_positions[i]
                if any([app!=i for app in room if app is not None]):
                    room_out = len([p for p in room if p is None])               
                    room_in = len([p for p in self.rooms[ap] if p is None])
                    if self.path_is_clear(ap_pos, rooms_positions[ap]) and self.room_is_available(ap):
                        cost = (abs(rooms_positions[ap]-ap_pos)+1+room_out+room_in) * step_cost[ap]
                        new_scenario = deepcopy(self)
                        new_scenario.rooms[i][-room_out-1] = None
                        new_scenario.rooms[ap][-room_in] = ap
                        self._add_scenario(new_scenario, cost, open_scenarios, closed_scenarios)
                    for pos in self._reachable_hallway_positions(i):
                        cost = (abs(pos-ap_pos)+1+room_out) * step_cost[ap]
                        new_scenario = deepcopy(self)
                        new_scenario.rooms[i][-room_out-1] = None
                        new_scenario.hallway[pos] = ap
                        """print(self)
                        print("    |")
                        print("    V")
                        print(new_scenario)
                        print(f"pos: {pos}, ap_pos: {ap_pos}, room_out: {room_out}")
                        print(f"COST: {cost}")"""
                        self._add_scenario(new_scenario, cost, open_scenarios, closed_scenarios)
        for hw_idx, ap in enumerate(self.hallway):
                if ap is not None and self.room_is_available(ap) and self.path_is_clear(hw_idx, rooms_positions[ap], True):
                    room_in = len([p for p in self.rooms[ap] if p is None])
                    cost = (abs(hw_idx-rooms_positions[ap]) + room_in) * step_cost[ap]
                    new_scenario = deepcopy(self)
                    new_scenario.hallway[hw_idx] = None
                    new_scenario.rooms[ap][-room_in] = ap
                    self._add_scenario(new_scenario, cost, open_scenarios, closed_scenarios)
            
    def update_scenario(self, open_scenarios, closed_scenarios):
        if self in closed_scenarios:
            return
        if self in open_scenarios:
            if self.cost >= open_scenarios[open_scenarios.index(self)].cost:
                return
            else:
                open_scenario = open_scenarios[open_scenarios.index(self)]
                open_scenario.cost = self.cost
                open_scenario.prev = self.prev
                return
        open_scenarios.add(self)
            
    def is_end(self):
        for i,r in enumerate(self.rooms):
            if not all([ap==i for ap in r]):
                return False
        return True
    
    def nearest_to_solution(self):
        value = 0
        for i, r in enumerate(self.rooms):
            for ap in r:
                if ap==i:
                    value +=1
                else: 
                    break       
        res = 2 if value==0 else 1./value
        return res
        
    
    def find_best_solution(self):
        closed_scenarios = SortedSet(key=hash)
        open_scenarios = SortedListWithKey(key=lambda s: (s.nearest_to_solution(), s.cost))
        open_scenarios.add(self)
        end_scenario = None
        found = 0
        n_improving_solutions = None
        while True:
            scenario = next_scenario(open_scenarios, closed_scenarios, max_cost=1000000000 if end_scenario is None else end_scenario.cost)
            if scenario is None:
                break
            if scenario.is_end():
                #: {scenario.cost}, end_scenario.cost: {None if not end_scenario else end_scenario.cost}")
                if end_scenario is None or scenario.cost < end_scenario.cost:
                    end_scenario = scenario
                    found += 1
                    #print(f"found {found}-th end scenario for warmup with cost:{scenario.cost}")
                closed_scenarios.remove(end_scenario)
                if n_improving_solutions is not None and found==n_improving_solutions:
                    closed_scenarios.remove(self)
                    return end_scenario
            elif end_scenario is None or scenario.cost < end_scenario.cost:
                scenario.add_possible_outcomes(open_scenarios, closed_scenarios)
        #print(f"found {found} end scenarios. minimum cost: {end_scenario.cost}")
        return end_scenario
        
            
    def get_path(self):
        path = [self]
        el = self.prev
        while el is not None:
            path.append(el.prev)
            el = el.prev
        return path[::-1]
            

def next_scenario(open_scenarios: SortedListWithKey[Burrow], closed_scenarios: SortedSet[Burrow], max_cost = 1000000):
    for next_node in open_scenarios:
        if next_node.cost < max_cost:
            open_scenarios.remove(next_node)
            closed_scenarios.add(next_node)
            return next_node
    return None            

with open("input.txt", "r") as f:
    lines = f.read().splitlines()
    rooms = [[lines[3][3], lines[2][3]], [lines[3][5], lines[2][5]], [lines[3][7], lines[2][7]], [lines[3][9], lines[2][9]]]
    rooms = list(map(lambda r: list(map(lambda x: names_to_indices[x], r)), rooms))
    starting_burrow = Burrow(rooms)

end_scenario = starting_burrow.find_best_solution()
print(f"answer_1: {end_scenario.cost}")


print("\n".join([str(p) for p in end_scenario.get_path()]))






answer_1: 15358
None
#############
#...........#
###2#0#1#3###
###3#2#0#1###
  ######### 
cost: 0
#############
#.......1...#
###2#0#.#3###
###3#2#0#1###
  ######### 
cost: 20
#############
#0......1...#
###2#0#.#3###
###3#2#.#1###
  ######### 
cost: 28
#############
#0......1...#
###.#0#.#3###
###3#2#2#1###
  ######### 
cost: 728
#############
#00.....1...#
###.#.#.#3###
###3#2#2#1###
  ######### 
cost: 732
#############
#00.....1...#
###.#.#2#3###
###3#.#2#1###
  ######### 
cost: 1232
#############
#00.........#
###.#.#2#3###
###3#1#2#1###
  ######### 
cost: 1282
#############
#00.......3.#
###.#.#2#.###
###3#1#2#1###
  ######### 
cost: 3282
#############
#00.......3.#
###.#1#2#.###
###3#1#2#.###
  ######### 
cost: 3352
#############
#00.........#
###.#1#2#.###
###3#1#2#3###
  ######### 
cost: 6352
#############
#00.3.......#
###.#1#2#.###
###.#1#2#3###
  ######### 
cost: 9352
#############
#0..3.......#
###.#1#2#.###
###0#1#2#3###
  ######### 
cost: 9355
#############
#...........#


In [371]:
with open("input.txt", "r") as f:
    lines = f.read().splitlines()
    rooms = [[lines[3][3], lines[2][3]], [lines[3][5], lines[2][5]], [lines[3][7], lines[2][7]], [lines[3][9], lines[2][9]]]
    rooms = list(map(lambda r: list(map(lambda x: names_to_indices[x], r)), rooms))
    starting_burrow = Burrow(rooms)

rooms_2 = deepcopy(rooms)
rooms_2[0].insert(1,3)
rooms_2[0].insert(1,3)
rooms_2[1].insert(1,2)
rooms_2[1].insert(1,1)
rooms_2[2].insert(1,1)
rooms_2[2].insert(1,0)
rooms_2[3].insert(1,0)
rooms_2[3].insert(1,2)
starting_burrow_2 = Burrow(rooms_2)
end_scenario = starting_burrow_2.find_best_solution()
print(f"answer_2: {end_scenario.cost}")

answer_2: 51436
